<a href="https://colab.research.google.com/github/royanlord/DL_Image-Classification_Pneumonia/blob/main/DL_Pneumonia_Deepseek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


latihan yang dipelajari meliputi:
    1. preprocessing
    2. load gambar dalam sebuah folder
    3. spliting dataset train test
    4. klasifikasi model evaluasi
    5. Find best parameter

In [1]:
# --- 1.1 Instalasi dan Konfigurasi Kaggle ---
!pip install kaggle opendatasets
!pip install opencv-python tensorflow scikit-learn

# Mengunggah kaggle.json
from google.colab import files
import os

if not os.path.exists("/root/.kaggle"):
    os.makedirs("/root/.kaggle")

# Upload file kaggle.json Anda
print("Silakan unggah file kaggle.json Anda:")
uploaded = files.upload()

# Pindahkan file ke direktori konfigurasi Kaggle
for fn in uploaded.keys():
    !mv "$fn" "/root/.kaggle/$fn"
    !chmod 600 /root/.kaggle/$fn

print("Konfigurasi Kaggle selesai.")

Silakan unggah file kaggle.json Anda:


Saving kaggle.json to kaggle.json
Konfigurasi Kaggle selesai.


In [2]:
# --- 1.2 Impor Pustaka ---
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.applications import Xception # Feature Extractor
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.utils import class_weight
import numpy as np
import cv2
import os
import math
import shutil

# --- 1.3 Unduh Dataset Baseline (Pediatrik Guangzhou) ---
# Gunakan identifier dataset Kaggle
kaggle_url = 'paultimothymooney/chest-xray-pneumonia'
print(f"Mengunduh dataset dari Kaggle: {kaggle_url}")

# Menggunakan opendatasets (membutuhkan otentikasi Kaggle di atas)
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

# Ekstrak file zip
!unzip -q chest-xray-pneumonia.zip -d chest_xray_data

# Konfigurasi Global dan Path
IMG_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 2

# Path Dataset (pastikan sesuai dengan struktur setelah unzip)
BASE_DIR = 'chest_xray_data/chest_xray'
TRAIN_DIR = os.path.join(BASE_DIR, 'train')
VAL_DIR = os.path.join(BASE_DIR, 'val')
TEST_DIR = os.path.join(BASE_DIR, 'test')

# Pemeriksaan Data
print(f"\nDataset Train Normal: {len(os.listdir(os.path.join(TRAIN_DIR, 'NORMAL')))} citra")
print(f"Dataset Train Pneumonia: {len(os.listdir(os.path.join(TRAIN_DIR, 'PNEUMONIA')))} citra")

print(f"\nDirektori Training: {TRAIN_DIR}")
print(f"Direktori Validation: {VAL_DIR}")
print(f"Direktori Test: {TEST_DIR}")

Mengunduh dataset dari Kaggle: paultimothymooney/chest-xray-pneumonia
Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
 99% 2.27G/2.29G [00:29<00:00, 89.3MB/s]
100% 2.29G/2.29G [00:29<00:00, 82.7MB/s]

Dataset Train Normal: 1341 citra
Dataset Train Pneumonia: 3875 citra


In [9]:
# ==============================================
# PERBAIKAN 1: FIX VALIDATION SET YANG KECIL
# ==============================================
print("\n" + "="*60)
print("MEMPERBAIKI VALIDATION SET (HANYA 16 SAMPLE)")
print("="*60)

from sklearn.model_selection import train_test_split # Import train_test_split

def create_proper_validation_set():
    """Buat validation set yang proper dari training data"""

    # Backup validation asli jika ada
    if os.path.exists(VAL_DIR):
        backup_dir = VAL_DIR + "_backup"
        if not os.path.exists(backup_dir):
            shutil.move(VAL_DIR, backup_dir)
            print(f"Validation set asli di-backup ke: {backup_dir}")

    # Buat direktori validation baru
    for class_name in ['NORMAL', 'PNEUMONIA']:
        os.makedirs(os.path.join(VAL_DIR, class_name), exist_ok=True)

    total_moved = 0
    for class_name in ['NORMAL', 'PNEUMONIA']:
        train_class_dir = os.path.join(TRAIN_DIR, class_name)
        val_class_dir = os.path.join(VAL_DIR, class_name)

        # Dapatkan semua file training
        all_files = [f for f in os.listdir(train_class_dir)
                    if f.lower().endswith(('.jpeg', '.jpg', '.png', '.jfif'))]

        print(f"\nKelas {class_name}: {len(all_files)} file di training")

        # Ambil 15% untuk validation
        if len(all_files) > 0:
            train_files, val_files = train_test_split(
                all_files,
                test_size=0.15,  # 15% untuk validation
                random_state=42
            )

            print(f"  -> {len(val_files)} file akan dipindah ke validation")

            # Copy (bukan move) ke validation
            for file_name in val_files:
                src = os.path.join(train_class_dir, file_name)
                dst = os.path.join(val_class_dir, file_name)
                shutil.copy2(src, dst)  # Copy, biar training tidak berkurang banyak

            total_moved += len(val_files)

    return total_moved

# Jalankan perbaikan
total_val_images = create_proper_validation_set()
print(f"\n✅ Total images di validation set setelah perbaikan: {total_val_images}")


MEMPERBAIKI VALIDATION SET (HANYA 16 SAMPLE)

Kelas NORMAL: 1341 file di training
  -> 202 file akan dipindah ke validation

Kelas PNEUMONIA: 3875 file di training
  -> 582 file akan dipindah ke validation

✅ Total images di validation set setelah perbaikan: 784


In [10]:
# Cek Distribusi Data
def check_data_distribution():
    """Cek distribusi data di semua set"""

    print("\n" + "="*70)
    print("DISTRIBUSI DATA SETELAH PERBAIKAN")
    print("="*70)

    for dataset_name, dataset_dir in [("TRAINING", TRAIN_DIR),
                                     ("VALIDATION", VAL_DIR),
                                     ("TEST", TEST_DIR)]:

        normal_dir = os.path.join(dataset_dir, 'NORMAL')
        pneumonia_dir = os.path.join(dataset_dir, 'PNEUMONIA')

        # Hitung file
        normal_count = len([f for f in os.listdir(normal_dir)
                           if f.lower().endswith(('.jpeg', '.jpg', '.png', '.jfif'))]) if os.path.exists(normal_dir) else 0
        pneumonia_count = len([f for f in os.listdir(pneumonia_dir)
                              if f.lower().endswith(('.jpeg', '.jpg', '.png', '.jfif'))]) if os.path.exists(pneumonia_dir) else 0

        total = normal_count + pneumonia_count

        print(f"\n{dataset_name}:")
        print(f"  NORMAL: {normal_count} images")
        print(f"  PNEUMONIA: {pneumonia_count} images")
        print(f"  TOTAL: {total} images")
        if total > 0:
            print(f"  Rasio NORMAL: {normal_count/total:.1%}")
            print(f"  Rasio PNEUMONIA: {pneumonia_count/total:.1%}")

check_data_distribution()


DISTRIBUSI DATA SETELAH PERBAIKAN

TRAINING:
  NORMAL: 1341 images
  PNEUMONIA: 3875 images
  TOTAL: 5216 images
  Rasio NORMAL: 25.7%
  Rasio PNEUMONIA: 74.3%

VALIDATION:
  NORMAL: 202 images
  PNEUMONIA: 582 images
  TOTAL: 784 images
  Rasio NORMAL: 25.8%
  Rasio PNEUMONIA: 74.2%

TEST:
  NORMAL: 234 images
  PNEUMONIA: 390 images
  TOTAL: 624 images
  Rasio NORMAL: 37.5%
  Rasio PNEUMONIA: 62.5%


In [11]:
# --- 2.1 Fungsi Peningkatan Citra (CLAHE) ---
def apply_clahe(img):
    """Menerapkan CLAHE dan mengkonversikannya kembali ke 3-channel (RGB)."""

    if isinstance(img, tf.Tensor):
        img_array = img.numpy().astype(np.uint8)
    else:
        # Konversi ke uint8 jika diperlukan (asumsi input dari generator adalah 0-255 atau float)
        img_array = (np.array(img) * 255).astype(np.uint8) if img.dtype != np.uint8 else np.array(img)

    # Konversi ke Grayscale
    if img_array.ndim == 3 and img_array.shape[-1] == 3:
        gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
    else:
        gray = img_array

    # Terapkan CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    clahe_img = clahe.apply(gray)

    # Konversi kembali ke 3-channel (RGB)
    clahe_rgb = cv2.cvtColor(clahe_img, cv2.COLOR_GRAY2RGB)

    return clahe_rgb

In [12]:
# --- 2.2 Custom Preprocessing dan Data Generator ---
def custom_preprocessing(img):
    """Pipeline Preprocessing: CLAHE -> Normalisasi -> Preprocessing Xception."""

    img_clahe = apply_clahe(img)

    # Normalisasi (0-1)
    img_normalized = img_clahe / 255.0

    # Preprocessing Xception (didefinisikan di sini untuk Keras API)
    return tf.keras.applications.xception.preprocess_input(img_normalized * 255.0)

# Definisi Generator
train_datagen = ImageDataGenerator(
    preprocessing_function=custom_preprocessing,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
)

# Hanya preprocessing untuk validation dan test
val_test_datagen = ImageDataGenerator(
    preprocessing_function=custom_preprocessing
)

# Iterator Data
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

val_generator = val_test_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,  # Gunakan batch size yang sama
    class_mode='categorical',
    shuffle=True,  # JANGAN SHUFFLE validation!
    seed=42
)

test_generator = val_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

print(f"\n✅ Data generators berhasil dibuat:")
print(f"   Training samples: {train_generator.samples}")
print(f"   Validation samples: {val_generator.samples}")
print(f"   Test samples: {test_generator.samples}")

Found 5216 images belonging to 2 classes.
Found 784 images belonging to 2 classes.
Found 624 images belonging to 2 classes.

✅ Data generators berhasil dibuat:
   Training samples: 5216
   Validation samples: 784
   Test samples: 624


In [13]:
# Validasi Data Generators

def validate_generators():
    """Validasi bahwa generators bekerja dengan benar"""

print("\n" + "="*70)
print("VALIDASI DATA GENERATORS")
print("="*70)

# Ambil satu batch dari training
x_train_batch, y_train_batch = next(train_generator)
train_labels = np.argmax(y_train_batch, axis=1)

print(f"\nTraining Batch:")
print(f"  Batch shape: {x_train_batch.shape}")
print(f"  Label shape: {y_train_batch.shape}")
print(f"  Label values (5 pertama): {train_labels[:5]}")
print(f"  Unique labels in batch: {np.unique(train_labels)}")

# Reset dan ambil dari validation
val_generator.reset()
x_val_batch, y_val_batch = next(val_generator)
val_labels = np.argmax(y_val_batch, axis=1)

print(f"\nValidation Batch:")
print(f"  Batch shape: {x_val_batch.shape}")
print(f"  Label shape: {y_val_batch.shape}")
print(f"  Label values: {val_labels}")
print(f"  Unique labels in batch: {np.unique(val_labels)}")

# Cek apakah ada kedua class
if len(np.unique(val_labels)) < 2:
    print("\n⚠️ PERINGATAN: Validation batch hanya memiliki satu class!")
    print("   Solusi: Cek apakah validation set memiliki kedua class")

    # Cek distribusi di validation set
    val_normal = len(os.listdir(os.path.join(VAL_DIR, 'NORMAL')))
    val_pneumonia = len(os.listdir(os.path.join(VAL_DIR, 'PNEUMONIA')))
    print(f"   NORMAL di validation: {val_normal}")
    print(f"   PNEUMONIA di validation: {val_pneumonia}")

    if val_normal == 0 or val_pneumonia == 0:
        print("\n❌ ERROR: Validation set tidak memiliki kedua class!")
        print("   Tambahkan file ke folder validation yang kosong")
else:
    print("\n✅ SUCCESS: Validation batch memiliki kedua class!")

VALIDASI DATA GENERATORS

Training Batch:
  Batch shape: (32, 224, 224, 3)
  Label shape: (32, 2)
  Label values (5 pertama): [1 0 1 1 1]
  Unique labels in batch: [0 1]

Validation Batch:
  Batch shape: (32, 224, 224, 3)
  Label shape: (32, 2)
  Label values: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  Unique labels in batch: [0]

⚠️ PERINGATAN: Validation batch hanya memiliki satu class!
   Solusi: Cek apakah validation set memiliki kedua class
   NORMAL di validation: 202
   PNEUMONIA di validation: 582


In [7]:
# --- 2.3 Perhitungan Class Weights ---
class_labels = train_generator.classes
class_weights_array = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(class_labels),
    y=class_labels
)
class_weights_dict = dict(enumerate(class_weights_array))

print(f"\nClass Weights:")
print(f"  NORMAL (0): {class_weights_dict[0]:.4f}")
print(f"  PNEUMONIA (1): {class_weights_dict[1]:.4f}")

validate_generators()


Bobot Kelas (0=NORMAL, 1=PNEUMONIA): {0: np.float64(1.9448173005219984), 1: np.float64(0.6730322580645162)}


In [ ]:
# --- 3.1 Definisi Model (Enhanced PneuX-Net) ---
def build_enhanced_pneumo_net(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=2):

    # 1. Feature Extractor (Xception)
    base_xception = Xception(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    # Freeze the base layers for the first phase
    for layer in base_xception.layers:
        layer.trainable = False

    inputs = Input(shape=input_shape)
    x = base_xception(inputs, training=False)
    x = GlobalAveragePooling2D()(x)

    # 2. Classification Head (EfficientNet-inspired replacement for KNC)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)

    # 3. Output Layer
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs, name="Enhanced_PneuX_Net")

    return model

model = build_enhanced_pneumo_net()
model.summary()

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "Enhanced_PneuX_Net"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ xception (Functional)           │ (None, 7, 7, 2048)     │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,976,490 (83.83 MB)

 Trainable params: 1,115,010 (4.25 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

In [ ]:
# Setting Callback
# 1. Model Checkpoint - simpan model terbaik
checkpoint_callback = ModelCheckpoint(
    filepath='best_model_phase1.h5',
    monitor='val_accuracy',  # Monitor accuracy
    mode='max',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

# 2. Early Stopping - lebih sabar
early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=10,  # 10 epoch sebelum stop
    min_delta=0.001,
    restore_best_weights=True,
    verbose=1
)

# 3. Reduce Learning Rate on Plateau
reduce_lr_callback = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
    verbose=1
)

# 4. CSV Logger untuk tracking
csv_logger = tf.keras.callbacks.CSVLogger('training_log.csv')

# List semua callbacks
callbacks_list = [
    checkpoint_callback,
    early_stopping_callback,
    reduce_lr_callback,
    csv_logger
]

In [ ]:
# --- 3.2 Pelatihan (Fase I) ---
LEARNING_RATE_PHASE1 = 1e-4
EPOCHS_PHASE1 = 30

model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE_PHASE1),
    loss='categorical_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'), # Metrik Klinis
        tf.keras.metrics.AUC(name='auc')        # Metrik Klinis
    ]
)

print("\n--- Memulai Pelatihan Fase 1: Feature Extraction Training ---")
history = model.fit(
    train_generator,
    epochs=EPOCHS_PHASE1,
    validation_data=val_generator,
    class_weight=class_weights_dict,
    callbacks=[model_checkpoint_callback, early_stopping_callback]
)

# Load bobot terbaik
model.load_weights(checkpoint_filepath)


--- Memulai Pelatihan Fase 1: Feature Extraction Training ---
Epoch 1/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 137s 712ms/step - accuracy: 0.8318 - auc: 0.8968 - loss: 0.4083 - precision: 0.8318 - recall: 0.8318 - val_accuracy: 0.5000 - val_auc: 0.4805 - val_loss: 1.9783 - val_precision: 0.5000 - val_recall: 0.5000
Epoch 2/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 109s 667ms/step - accuracy: 0.8388 - auc: 0.9089 - loss: 0.3765 - precision: 0.8388 - recall: 0.8388 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 1.7171 - val_precision: 0.5000 - val_recall: 0.5000
Epoch 3/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 108s 661ms/step - accuracy: 0.8422 - auc: 0.9103 - loss: 0.3723 - precision: 0.8422 - recall: 0.8422 - val_accuracy: 0.5000 - val_auc: 0.4453 - val_loss: 2.5358 - val_precision: 0.5000 - val_recall: 0.5000
Epoch 4/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 109s 669ms/step - accuracy: 0.8402 - auc: 0.9096 - loss: 0.3764 - precision: 0.8402 - recall: 0.8402 - val_accuracy: 0.5000 - val_auc: 0.4922 - val_loss: 2.401

In [ ]:
# Evaluasi Fase 1
def evaluate_phase1():
    """Evaluasi hasil fase 1 sebelum melanjutkan ke fine-tuning"""
    print("\n" + "="*60)
    print("EVALUASI HASIL FASE 1")
    print("="*60)

    # Plot training history
    plt.figure(figsize=(15, 5))

    # Plot accuracy
    plt.subplot(1, 3, 1)
    plt.plot(history_phase1.history['accuracy'], label='Training Accuracy')
    plt.plot(history_phase1.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Plot loss
    plt.subplot(1, 3, 2)
    plt.plot(history_phase1.history['loss'], label='Training Loss')
    plt.plot(history_phase1.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Plot AUC
    plt.subplot(1, 3, 3)
    plt.plot(history_phase1.history['auc'], label='Training AUC')
    plt.plot(history_phase1.history['val_auc'], label='Validation AUC')
    plt.title('Model AUC')
    plt.xlabel('Epoch')
    plt.ylabel('AUC')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Print metrics terbaik
    best_val_acc = max(history_phase1.history['val_accuracy'])
    best_val_auc = max(history_phase1.history['val_auc'])
    final_train_acc = history_phase1.history['accuracy'][-1]

    print(f"\n📊 METRIK TERBAIK FASE 1:")
    print(f"   Best Validation Accuracy: {best_val_acc:.4f}")
    print(f"   Best Validation AUC: {best_val_auc:.4f}")
    print(f"   Final Training Accuracy: {final_train_acc:.4f}")

    # Cek apakah fase 1 berhasil
    if best_val_acc < 0.65:
        print("\n⚠️  PERINGATAN: Validation accuracy fase 1 < 65%")
        print("   Mungkin perlu tuning hyperparameter atau periksa data")
        return False
    else:
        print("\n✅ FASE 1 BERHASIL! Bisa lanjut ke fine-tuning")
        return True

# Jalankan evaluasi
phase1_success = evaluate_phase1()

In [ ]:
# Hanya lanjut ke fase 2 jika fase 1 berhasil
if phase1_success:
    # --- 3.3 Fine-Tuning (Fase II) ---
    LEARNING_RATE_PHASE2 = 1e-5

    print("\n" + "="*60)
    print("MEMPERSIAPKAN FINE-TUNING (FASE 2)")
    print("="*60)

    # Unfreeze lapisan Xception teratas (50 layer terakhir)
    base_model = model.layers[1]
    for layer in base_model.layers[-50:]:  # Unfreeze 50 layer terakhir
        layer.trainable = True

    print(f"Trainable layers setelah unfreeze: {sum([l.trainable for l in base_model.layers])}/{len(base_model.layers)}")

    # Kompilasi ulang dengan Learning Rate yang lebih rendah
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE_PHASE2),
        loss='categorical_crossentropy',
        metrics=['accuracy', 'precision', 'recall', 'auc']
    )

    # Callbacks untuk fase 2
    checkpoint_phase2 = ModelCheckpoint(
        filepath='best_model_phase2.h5',
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    )

    early_stopping_phase2 = EarlyStopping(
        monitor='val_loss',
        patience=7,
        min_delta=0.001,
        restore_best_weights=True,
        verbose=1
    )

    print("\n" + "="*60)
    print("MEMULAI PELATIHAN FASE 2: FINE-TUNING")
    print("="*60)

    # Hitung initial epoch untuk fase 2
    initial_epoch_phase2 = len(history_phase1.history['loss'])
    total_epochs_phase2 = initial_epoch_phase2 + 15  # 15 epoch tambahan

    history_phase2 = model.fit(
        train_generator,
        epochs=total_epochs_phase2,
        initial_epoch=initial_epoch_phase2,
        validation_data=val_generator,
        class_weight=class_weights_dict,
        callbacks=[checkpoint_phase2, early_stopping_phase2, reduce_lr_callback],
        verbose=1
    )

    # Load bobot terbaik fase 2
    print("\nMemuat bobot terbaik dari fase 2...")
    model.load_weights('best_model_phase2.h5')

    # Simpan model final
    model.save('enhanced_pneumo_net_final.keras')
    print("Model final disimpan sebagai: enhanced_pneumo_net_final.keras")
else:
    print("\n❌ Tidak melanjutkan ke fase 2 karena fase 1 belum optimal")
    print("   Perbaiki fase 1 terlebih dahulu")


--- Memulai Pelatihan Fase 2: Fine-Tuning ---
Epoch 7/30
163/163 ━━━━━━━━━━━━━━━━━━━━ 139s 723ms/step - accuracy: 0.8105 - auc: 0.8801 - loss: 0.4772 - precision: 0.8105 - recall: 0.8105 - val_accuracy: 0.5000 - val_auc: 0.5586 - val_loss: 1.1581 - val_precision: 0.5000 - val_recall: 0.5000
Epoch 8/30
163/163 ━━━━━━━━━━━━━━━━━━━━ 112s 689ms/step - accuracy: 0.8303 - auc: 0.8956 - loss: 0.3984 - precision: 0.8303 - recall: 0.8303 - val_accuracy: 0.5000 - val_auc: 0.5703 - val_loss: 1.0174 - val_precision: 0.5000 - val_recall: 0.5000
Epoch 9/30
163/163 ━━━━━━━━━━━━━━━━━━━━ 113s 692ms/step - accuracy: 0.8537 - auc: 0.9145 - loss: 0.3628 - precision: 0.8537 - recall: 0.8537 - val_accuracy: 0.5000 - val_auc: 0.5430 - val_loss: 1.0724 - val_precision: 0.5000 - val_recall: 0.5000
Epoch 10/30
163/163 ━━━━━━━━━━━━━━━━━━━━ 113s 695ms/step - accuracy: 0.8568 - auc: 0.9253 - loss: 0.3397 - precision: 0.8568 - recall: 0.8568 - val_accuracy: 0.4375 - val_auc: 0.5391 - val_loss: 1.0035 - val_precisi

In [ ]:
# --- 4.1 Evaluasi Internal ---
print("\n--- Evaluasi Akhir pada Dataset Test Internal (Guangzhou) ---")
results_internal = model.evaluate(test_generator, verbose=1)

print(f"\n📊 HASIL EVALUASI TEST SET:")
print(f"   Loss Test: {results_internal[0]:.4f}")
print(f"   Accuracy Test: {results_internal[1]:.4f}")
print(f"   Precision Test: {results_internal[2]:.4f}")
print(f"   Recall Test: {results_internal[3]:.4f}")
print(f"   AUC Test: {results_internal[4]:.4f}")

# Simpan hasil untuk visualisasi plot
# Plot akurasi dan loss history


--- Evaluasi Akhir pada Dataset Test Internal (Guangzhou) ---
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 272ms/step - accuracy: 0.3806 - auc: 0.3986 - loss: 1.1820 - precision: 0.3806 - recall: 0.3806

Loss Test: 0.7155
Accuracy Test: 0.6458
Recall Test: 0.6458
AUC Test: 0.7391
